In [1]:
import os
import mne
import pycartool.io
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt

import umap
from features import *
from my_io import *

c:\Users\ferat\.conda\envs\ep\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\ferat\.conda\envs\ep\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\ferat\.conda\envs\ep\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The '

## Read EEG data

In [2]:
def read_epiliptic_events(file, sfreq):
    df = pd.read_csv(file, sep="\t", skiprows=1, names=['start', 'stop', 'label'])
    df['start_time'] = df['start'] / sfreq
    df['stop_time'] = df['stop'] / sfreq
    df['duration'] = df['stop_time'] - df['start_time']
    df['label'] = [l.split('_')[0] for l in df['label'].values]
    annotations = mne.Annotations(df['start_time'], df['duration'], df['label'])
    return(annotations)
file = fr'V:\switchdrive\Brainhack\KMR11\d17\Epileptic_events.mrk'

def read_file(fname):
    # Read Raw
    base_path = os.path.dirname(fname) 
    raw = read_sef(fname)
    # Read Bads
    bad_annotations = mne.Annotations(0, 0, 'null')
    for file in os.listdir(base_path):
        if file.lower().startswith('bad'):
            print(file)
            path = os.path.join(base_path, file)
            annotations = read_bad_file(path, raw.info['sfreq'])
            bad_annotations += annotations
    # Read epileptic
    epileptic_annotations = mne.Annotations(0, 0, 'null')
    for file in os.listdir(base_path):
        if file.lower().startswith('epileptic'):
            print(file)
            path = os.path.join(base_path, file)
            annotations = read_epiliptic_events(path, raw.info['sfreq'])
            epileptic_annotations += annotations
    # Read background
    background_annotations = mne.Annotations(0, 0, 'null')
    for file in os.listdir(base_path):
        if file.lower().endswith('bck.mrk'):
            print(file)
            path = os.path.join(base_path, file)
            annotations = read_background_events_file(path, raw.info['sfreq'])
            background_annotations += annotations
    annotations = epileptic_annotations + bad_annotations + background_annotations
    raw.set_annotations(annotations) 
    return(raw)

In [42]:
files = list()

subjects_folder = fr'Z:\Animal\Fabien\DATA\KMR'
for subject in os.listdir(subjects_folder):
    subject_folder = os.path.join(subjects_folder, subject)
    if os.path.isdir(subject_folder):
        for day in os.listdir(subject_folder):
            day_folder = os.path.join(subject_folder, day)
            if os.path.isdir(day_folder):
                for file in os.listdir(day_folder):
                    if file.endswith('.sef'):
                        file = os.path.join(day_folder, file)
                        files.append(file)
files

['Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d11\\KMR1_d11_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d13\\KMR1_d13_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d15\\KMR1_d15_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d17\\KMR1_d17_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d19\\KMR1_d19_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d21\\KMR1_d21_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d23\\KMR1_d23_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d25\\KMR1_d25_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d27\\KMR1_d27_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d29\\KMR1_d29_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d3\\KMR1_d3_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d5\\KMR1_d5_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d7\\KMR1_d7_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d9\\KMR1_d9_Raw_DS.sef',
 'Z:\\Animal\\

In [44]:
def extract_features(file, epoch_duration, bands):
    try:
        subject = os.path.basename(file).split('_')[0]
        day = os.path.basename(file).split('_')[1] 

        features = []
        column_names = []

        raw = read_file(file)
        epochs = mne.make_fixed_length_epochs(raw, duration=epoch_duration, preload=True)
        
        for band in bands:                
            epoch_bands = epochs.copy().filter(band[0], band[1])
            for ch_name in epoch_bands.ch_names:
                epoch_band_channel = epoch_bands.copy().pick(ch_name)
                data = epoch_band_channel.get_data()

                # Extract Features
                activity_features = activity(data)
                features.append(activity_features)
                column_names += [f'{ch_name}_{band}_activity_feature_{i}' for i in range(activity_features.shape[-1])]

                mobility_features = mobility(data)
                features.append(mobility_features)
                column_names += [f'{ch_name}_{band}_mobility_feature_{i}' for i in range(mobility_features.shape[-1])]

                complexity_features = complexity(data)
                features.append(complexity_features)
                column_names += [f'{ch_name}_{band}_complexity_feature_{i}' for i in range(complexity_features.shape[-1])]

                time_features = extract_time_feat(data)
                time_features = time_features.reshape((time_features.shape[0],-1))
                features.append(time_features)
                column_names += [f'{ch_name}_{band}_time_feature_{i}' for i in range(time_features.shape[-1])]

                frequency_features = extract_freq_feat(data, sfreq=epochs.info['sfreq'])
                frequency_features = frequency_features.reshape((frequency_features.shape[0],-1))
                features.append(frequency_features)
                column_names += [f'{ch_name}_{band}_frequency_feature_{i}' for i in range(frequency_features.shape[-1])]

                information_features = extract_information_feat(data, sfreq=epochs.info['sfreq'])
                information_features = information_features.reshape((information_features.shape[0],-1))
                features.append(information_features)
                column_names += [f'{ch_name}_{band}_information_feature_{i}' for i in range(information_features.shape[-1])]

                dwt_features = extract_dwt_feat(data)
                dwt_features = dwt_features.reshape((dwt_features.shape[0],-1))
                features.append(dwt_features)
                column_names += [f'{ch_name}_{band}_dwt_feature_{i}' for i in range(dwt_features.shape[-1])]

            #events_ = np.array([list(events_id.keys())[list(events_id.values()).index(event)] for event in epochs.events[:,2]]).reshape(-1,1)
            #features.append(events_)
            #column_names += ['event_name']

            ts = epochs.events[:,0].reshape(-1,1) / raw.info['sfreq']
            features.append(ts)
            column_names += ['start']

            days = np.array([day] * len(epochs)).reshape(-1,1)
            features.append(days)
            column_names += ['day']

            subjects = np.array([subject] * len(epochs)).reshape(-1,1)
            features.append(subjects)
            column_names += ['subject']
        
            features = np.hstack(features)
            df = pd.DataFrame(features, columns=column_names)
            return(df)
    except Exception as e:
        print(e)
        return(e)

In [5]:
from joblib import Parallel, delayed

# Config
epoch_duration = 1
bands = [(1,30), (80,200), (250,500)]

r = Parallel(n_jobs=10)(delayed(extract_features)(file, epoch_duration, bands) for file in files[:8])

In [19]:
r[2]['day']

0      KMR
1      KMR
2      KMR
3      KMR
4      KMR
      ... 
577    KMR
578    KMR
579    KMR
580    KMR
581    KMR
Name: day, Length: 582, dtype: object

In [7]:
df_col_merged = pd.concat(r, axis=1)

In [13]:
df_col_merged.to_csv('test.tsv', sep="\t")

In [23]:
df = pd.DataFrame(np.vstack(all_features), columns=column_names)
df['code'] =  pd.Categorical(df.event_name).codes
features = [column for column in df.columns if "feature" in column]
non_features = [column for column in df.columns if not "feature" in column]

df[features].to_csv('features.tsv',sep='\t', index=False, header=False)
df[non_features].to_csv('non_features.tsv',sep='\t')

In [24]:
fit = umap.UMAP(n_neighbors=15, n_components=2)
data = df[features].values

In [26]:
u = fit.fit_transform(data)
#df['data'] = [data for data in np.vstack(datas)[:,0,:]]
df['x1'] = u[:,0].reshape(-1)
df['x2'] = u[:,1].reshape(-1)
plt.scatter(u[:,0], u[:,1], c=df['code'] , s=1)
plt.legend()

c:\Users\ferat\.conda\envs\ep\lib\site-packages\sklearn\manifold\_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [29]:
import umap.plot 
umap.plot.output_notebook()


p = umap.plot.interactive(fit, labels=df['code'],
                          hover_data=df[non_features],
                          point_size=4,
                          theme='fire',
                          background='black',
                          #color_key= ['FR', 'HAHF', 'HALF', 'LAHF', 'LALF', 'RP', 'background', 'null'],
                          interactive_text_search_columns=True)
#
umap.plot.show(p)

c:\Users\ferat\.conda\envs\ep\lib\site-packages\umap\plot.py:203: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()


Loading BokehJS ...

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.layouts import row
from bokeh.palettes import brewer
from bokeh.io import  output_notebook
output_notebook()

colors = brewer["Spectral"][len(df.code.unique())]
colormap = {i: colors[i] for i in df.code.unique()}
colors = [colormap[x] for x in df.code]
df['color'] = colors

tooltips = [
    ("day", "@day"),
    ("event", "@event_name"),
    ("subject", "@subject")]

s1 = ColumnDataSource(data=df[['x1','x2','day', 'event_name', 'subject', 'data', 'code', 'color']])
p1 = figure(width=400, height=400, tools='tap,hover,pan,wheel_zoom,box_zoom,reset', title="UMAP",
            tooltips=tooltips)
p1.scatter('x1', 'x2', source=s1, color='color')

df2 = pd.DataFrame()
df2['x'] =  np.arange(0,len(df['data'][0]))
df2['y'] = df['data'].values[0]
s2 = ColumnDataSource(data=df2)
p2 = figure(width=400, height=400, title="Data")
p2.line('x', 'y', source=s2)

s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2), code="""
        const inds = cb_obj.indices;
        console.log(inds[0]);
        const d2 = s2.data;
        console.log(s1.data.data);
        d2['x'] = []
        d2['y'] = []
        for (let i = 0; i < d2.index.length; i++) {
            d2['x'].push(i)
            d2['y'].push(s1.data.data[inds[0]][i])
        }
        s2.change.emit();
    """)
)


#layout = row(p1, p2)
show(row(p1, p2))
#show(p2)

In [ ]:
from bokeh.palettes import brewer
from bokeh.io import  output_notebook
output_notebook()

colors = brewer["Spectral"][len(df.code.unique())]
colormap = {i: colors[i] for i in df.code.unique()}
colors = [colormap[x] for x in df.code]
df['color'] = colors

tooltips = [
    ("day", "$day"),
    ("event", "$event_name"),
    ("subject", "$subject")]

s1 = ColumnDataSource(data=df[['x1','x2','day', 'event_name', 'subject', 'data', 'code', 'color']])
p1 = figure(width=400, height=400, tools='tap,hover,pan,wheel_zoom,box_zoom,reset', title="UMAP",
            tooltips=tooltips)
p1.scatter('x1', 'x2', source=s1)
show(p1)

In [ ]:
day = 23
subject = 11
start = 1656
raw = read_file(fr'V:\\switchdrive\\Brainhack\\KMR{subject}\\d{day}\\KMR{subject}_d{day}_Raw_DS.Avg_ref.sef')

for band in bands:
    raw_ = raw.copy().filter(band[0], band[1])
    raw_.plot(scalings='auto', start=start, decim=1)